1) **Derives a new table of US Customers called USACusts.** notice here we're using the derived table recieved from a subquery in our FROM clause.

In [2]:
use Northwinds2022TSQLV7

select *
from (
    select Customerid, CustomerCompanyName
    from Sales.Customer
    where CustomerCountry = N'USA'
) as USACusts

(13 rows affected)

Total execution time: 00:00:00.018

Customerid,CustomerCompanyName
32,Customer YSIQX
36,Customer LVJSO
43,Customer UISOJ
45,Customer QXPPT
48,Customer DVFMB
55,Customer KZQZT
65,Customer NYUHS
71,Customer LCOUJ
75,Customer XOJYP
77,Customer LCYBZ


2) **Derives a table for each years number of customers.** Makes use of inline aliasing, take a peek at comments.

In [13]:
use Northwinds2022TSQLV7

-- select YEAR(OrderDate) as orderyear,
--     COUNT(distinct CustomerId) as numcusts
-- from Sales.[Order]
-- group by orderyear --obv we'll get a parsing error

select orderyear,
    COUNT(distinct custid) as numcusts
from (
    --years each customer has ordered
    select YEAR(OrderDate) as orderyear,
        CustomerId as custid --these lines will execute before our first statement so hence the aliasing is possible
    from Sales.[Order]
) as D
group by orderyear

(3 rows affected)

Total execution time: 00:00:00.026

orderyear,numcusts
2014,67
2015,86
2016,81


3) **External column aliasing, does same as above.**

In [16]:
use Northwinds2022TSQLV7

select orderyear, count(distinct custid) as numcusts
from (
    select year(OrderDate), CustomerId --notice we dont alias here
    from Sales.[Order] 
) as D (orderyear, custid) -- but here after we return the derived table!
group by orderyear

(3 rows affected)

Total execution time: 00:00:00.046

orderyear,numcusts
2014,67
2015,86
2016,81


4) **Yearly count of customers handled by Employee 3.** Uses arguments.

In [20]:
use Northwinds2022TSQLV7

DECLARE @empid AS INT = 3; --used as an argument for the table we derive

SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM (SELECT YEAR(OrderDate) AS orderyear, 
        CustomerId as custid
      FROM Sales.[Order]
      WHERE EmployeeId = @empid) AS D
GROUP BY orderyear;
GO

(3 rows affected)

Total execution time: 00:00:00.017

orderyear,numcusts
2014,16
2015,46
2016,30


5) **Years where number of customers exceeds 70.** Using nested derived tables for inline aliasing, notice how the top layer is easiest to read.

In [21]:
use Northwinds2022TSQLV7

SELECT orderyear, numcusts
FROM (SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
      FROM (SELECT YEAR(OrderDate) AS orderyear, 
                CustomerId as custid
            FROM Sales.[Order]) AS D1
      GROUP BY orderyear) AS D2
WHERE numcusts > 70;

(2 rows affected)

Total execution time: 00:00:00.033

orderyear,numcusts
2015,86
2016,81


6) **Same as above, but not as readable as you can see due to top level aliasing.**

In [22]:
use Northwinds2022TSQLV7

SELECT YEAR(OrderDate) AS orderyear, COUNT(DISTINCT CustomerId) AS numcusts
FROM Sales.[Order]
GROUP BY YEAR(OrderDate)
HAVING COUNT(DISTINCT CustomerId) > 70;

(2 rows affected)

Total execution time: 00:00:00.022

orderyear,numcusts
2015,86
2016,81


7. **Number of current customers, previous customer, and growth of customers for each year.** Using two subquery to create tables of current customers, previous customers, uses left join to combine row with certain criteria, preserves NULLS.

In [31]:
use Northwinds2022TSQLV7

SELECT Cur.orderyear, 
  Cur.numcusts AS curnumcusts, Prv.numcusts AS prvnumcusts,
  Cur.numcusts - Prv.numcusts AS growth
FROM (SELECT YEAR(OrderDate) AS orderyear,
        COUNT(DISTINCT CustomerId) AS numcusts
      FROM Sales.[Order]
      GROUP BY YEAR(OrderDate)) AS Cur
  LEFT OUTER JOIN --we use left join to preserve rows and nulls, so as to not discard
     (SELECT YEAR(OrderDate) AS orderyear,
        COUNT(DISTINCT CustomerId) AS numcusts
      FROM Sales.[Order]
      GROUP BY YEAR(OrderDate)) AS Prv
    ON Cur.orderyear = Prv.orderyear + 1;


(3 rows affected)

Total execution time: 00:00:00.036

orderyear,curnumcusts,prvnumcusts,growth
2014,67,NULL,NULL
2015,86,67,19
2016,81,86,-5


8. **Create USACusts as derived table using COMMON table expressions.**

In [24]:
use Northwinds2022TSQLV7;

WITH USACusts AS
(
  SELECT CustomerId, CustomerCompanyName
  FROM Sales.Customer
  WHERE CustomerCountry = N'USA'
)
SELECT * FROM USACusts;

(13 rows affected)

Total execution time: 00:00:00.027

CustomerId,CustomerCompanyName
32,Customer YSIQX
36,Customer LVJSO
43,Customer UISOJ
45,Customer QXPPT
48,Customer DVFMB
55,Customer KZQZT
65,Customer NYUHS
71,Customer LCOUJ
75,Customer XOJYP
77,Customer LCYBZ


9. **Yearly count of customers.** Table expressions and INLINE column aliasing.

In [26]:
use Northwinds2022TSQLV7;

WITH C AS
(
  SELECT YEAR(OrderDate) AS orderyear, 
        CustomerId as custid
  FROM Sales.[Order]
)
SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM C
GROUP BY orderyear;

(3 rows affected)

Total execution time: 00:00:00.030

orderyear,numcusts
2014,67
2015,86
2016,81


10) **Yearly count of customers.** Table expression and EXTERNAL column aliasing.

In [27]:
use Northwinds2022TSQLV7;

WITH C(orderyear, custid) AS
(
  SELECT YEAR(OrderDate), CustomerId
  FROM Sales.[Order]
)
SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM C
GROUP BY orderyear;

(3 rows affected)

Total execution time: 00:00:00.031

orderyear,numcusts
2014,67
2015,86
2016,81


11) **Yearly count of customers handled by employee 3.** Table expressions, inline column aliasing, and ARGUMENTS.

In [28]:
use Northwinds2022TSQLV7;

DECLARE @empid AS INT = 3;

WITH C AS
(
  SELECT YEAR(OrderDate) AS orderyear, 
        CustomerId as custid
  FROM Sales.[Order]
  WHERE EmployeeId = @empid
)
SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM C
GROUP BY orderyear;

(3 rows affected)

Total execution time: 00:00:00.034

orderyear,numcusts
2014,16
2015,46
2016,30


12. Multiple CTES. High readibility. Creates a table of order years for each customer. Derives a table with column aliasing, we use this in another CTE. Derivies a table of the number of unique customers for each year. We use the final derived table to easily return **years where the cumber of customers exceeds 70.**

In [33]:
use Northwinds2022TSQLV7;

WITH C1 AS
(
  SELECT YEAR(OrderDate) AS orderyear, 
    CustomerId as custid --readable aliasing
  FROM Sales.[Order]
),
C2 AS
(
  SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
  FROM C1
  GROUP BY orderyear
)
SELECT orderyear, numcusts
FROM C2
WHERE numcusts > 70;

(2 rows affected)

Total execution time: 00:00:00.033

orderyear,numcusts
2015,86
2016,81


13. Utilizing MULTIPLE references to **get each years current customers, previous customer, and growth.** Notice how the already named derived table can have more than one alias, to perform table operations like join.

In [35]:
use Northwinds2022TSQLV7;

WITH YearlyCount AS
(
  SELECT YEAR(OrderDate) AS orderyear,
    COUNT(DISTINCT CustomerId) AS numcusts
  FROM Sales.[Order]
  GROUP BY YEAR(OrderDate)
)
SELECT Cur.orderyear, 
  Cur.numcusts AS curnumcusts, Prv.numcusts AS prvnumcusts,
  Cur.numcusts - Prv.numcusts AS growth
FROM YearlyCount AS Cur -- the derived table has muliple references / aliases
  LEFT OUTER JOIN YearlyCount AS Prv 
    ON Cur.orderyear = Prv.orderyear + 1;

(3 rows affected)

Total execution time: 00:00:00.060

orderyear,curnumcusts,prvnumcusts,growth
2014,67,NULL,NULL
2015,86,67,19
2016,81,86,-5


14) **All employees who share the same manager as employee 2.**

In [37]:
use Northwinds2022TSQLV7;

WITH EmpsCTE AS
(
  SELECT EmployeeId as empid, 
        EmployeeManagerId as  mgrid, 
        EmployeeFirstName as firstname,
        EmployeeLastName as lastname
  FROM HumanResources.Employee
  WHERE EmployeeId = 2
  
  UNION ALL --union the multiple tables, unioning a table to an operation including itself!
  
  --use employee2 table and employees table to derive a new one
  SELECT C.EmployeeId, C.EmployeeManagerId, C.EmployeeFirstName, C.EmployeeLastName
  FROM EmpsCTE AS P
    INNER JOIN HumanResources.Employee AS C
      ON C.EmployeeManagerId = P.empid
)
SELECT empid, mgrid, firstname, lastname
FROM EmpsCTE;

(8 rows affected)

Total execution time: 00:00:00.025

empid,mgrid,firstname,lastname
2,1,Don,Funk
3,2,Judy,Lew
5,2,Sven,Mortensen
6,5,Paul,Suurs
7,5,Russell,King
9,5,Patricia,Doyle
4,3,Yael,Peled
8,3,Maria,Cameron


15. **Creates a USACusts view.** Think of a view as a virtual table, or the result of a query (select statement).

In [50]:
use TSQLV4;

DROP VIEW IF EXISTS Sales.USACusts;
GO
CREATE VIEW Sales.USACusts
AS

SELECT
  custid, companyname, contactname, contacttitle, address,
  city, region, postalcode, country, phone, fax
FROM Sales.Customers
WHERE country = N'USA';
GO

SELECT custid, companyname
FROM Sales.USACusts;
GO

Commands completed successfully.

Commands completed successfully.

(13 rows affected)

Total execution time: 00:00:00.054

custid,companyname
32,Customer YSIQX
36,Customer LVJSO
43,Customer UISOJ
45,Customer QXPPT
48,Customer DVFMB
55,Customer KZQZT
65,Customer NYUHS
71,Customer LCOUJ
75,Customer XOJYP
77,Customer LCYBZ


16. Using the order by clause in an outer query to **get the region ordered customers from USAcusts.** \*ORDER BY NOT ALLOWED IN VIEW.

In [51]:
use TSQLV4

SELECT custid, companyname, region
FROM Sales.USACusts

(13 rows affected)

Total execution time: 00:00:02.886

custid,companyname,region
32,Customer YSIQX,OR
36,Customer LVJSO,OR
43,Customer UISOJ,WA
45,Customer QXPPT,CA
48,Customer DVFMB,OR
55,Customer KZQZT,AK
65,Customer NYUHS,NM
71,Customer LCOUJ,ID
75,Customer XOJYP,WY
77,Customer LCYBZ,OR


17. Don't rely on TOP when using Views.

In [57]:
ALTER VIEW Sales.USACusts
AS

SELECT TOP (100) PERCENT
  custid, companyname, contactname, contacttitle,
  city, region, postalcode, country, phone, fax
FROM Sales.Customers
WHERE country = N'USA'
ORDER BY region;
GO

select custid, companyname, region
from Sales.USACusts
GO

Commands completed successfully.

(13 rows affected)

Total execution time: 00:00:00.025

custid
32
36
43
45
48
55
65
71
75
77


18) Do not rely on OFFSET FETCH, even if it works.

In [61]:
ALTER VIEW Sales.USACusts
AS

SELECT 
  custid, companyname, contactname, contacttitle, address,
  city, region, postalcode, country, phone, fax
FROM Sales.Customers
WHERE country = N'USA'
ORDER BY region
OFFSET 0 ROWS;
GO

SELECT custid, companyname, region
FROM Sales.USACusts;
GO

Commands completed successfully.

(13 rows affected)

Total execution time: 00:00:00.041

custid,companyname,region
55,Customer KZQZT,AK
45,Customer QXPPT,CA
71,Customer LCOUJ,ID
78,Customer NLTYP,MT
65,Customer NYUHS,NM
48,Customer DVFMB,OR
32,Customer YSIQX,OR
36,Customer LVJSO,OR
77,Customer LCYBZ,OR
82,Customer EYHKM,WA


19). **Encrypting the USACusts Table.** Prevents this table from being replicated as well.

In [67]:
ALTER VIEW Sales.USACusts
AS

SELECT
  custid, companyname, contactname, contacttitle,
  city, region, postalcode, country, phone, fax
FROM Sales.Customers
WHERE country = N'USA';
GO

SELECT OBJECT_DEFINITION(OBJECT_ID('Sales.USACusts'));
GO

ALTER VIEW Sales.USACusts WITH ENCRYPTION
AS

SELECT
  custid, companyname, contactname, contacttitle,
  city, region, postalcode, country, phone, fax
FROM Sales.Customers
WHERE country = N'USA';
GO

SELECT OBJECT_DEFINITION(OBJECT_ID('Sales.USACusts'));

EXEC sp_helptext 'Sales.USACusts';
GO

Commands completed successfully.

(1 row affected)

Commands completed successfully.

(1 row affected)

The text for object 'Sales.USACusts' is encrypted.

Total execution time: 00:00:00.045

(No column name)
"CREATE VIEW Sales.USACusts AS SELECT custid, companyname, contactname, contacttitle, city, region, postalcode, country, phone, fax FROM Sales.Customers WHERE country = N'USA';"


(No column name)
NULL


20. Schemabinding. Here view the error trying to modify the schema leads to, when we schemabind, it alerts us of dependant views and prevents use from altering the underlying schema that is binded to the view. So in this case I can change any Schema besides the Sales Schema.

In [72]:
ALTER VIEW Sales.USACusts WITH SCHEMABINDING
AS

SELECT
  custid, companyname, contactname, contacttitle, address,
  city, region, postalcode, country, phone, fax
FROM Sales.Customers
WHERE country = N'USA';
GO

ALTER TABLE Sales.Customers DROP COLUMN address;
GO

Commands completed successfully.

(91 rows affected)

Total execution time: 00:00:00.060

custid,companyname,contactname,contacttitle,address,city,region,postalcode,country,phone,fax
1,Customer NRZBB,"Allen, Michael",Sales Representative,Obere Str. 0123,Berlin,NULL,10092,Germany,030-3456789,030-0123456
2,Customer MLTDN,"Hassall, Mark",Owner,Avda. de la Constitución 5678,México D.F.,NULL,10077,Mexico,(5) 789-0123,(5) 456-7890
3,Customer KBUDE,"Strome, David",Owner,Mataderos 7890,México D.F.,NULL,10097,Mexico,(5) 123-4567,NULL
4,Customer HFBZG,"Cunningham, Conor",Sales Representative,7890 Hanover Sq.,London,NULL,10046,UK,(171) 456-7890,(171) 456-7891
5,Customer HGVLZ,"Higginbotham, Tom",Order Administrator,Berguvsvägen 5678,Luleå,NULL,10112,Sweden,0921-67 89 01,0921-23 45 67
6,Customer XHXJV,"Poland, Carole",Sales Representative,Forsterstr. 7890,Mannheim,NULL,10117,Germany,0621-67890,0621-12345
7,Customer QXVLA,"Bansal, Dushyant",Marketing Manager,"2345, place Kléber",Strasbourg,NULL,10089,France,67.89.01.23,67.89.01.24
8,Customer QUHWH,"Ilyina, Julia",Owner,"C/ Araquil, 0123",Madrid,NULL,10104,Spain,(91) 345 67 89,(91) 012 34 56
9,Customer RTXGC,"Raghav, Amritansh",Owner,"6789, rue des Bouchers",Marseille,NULL,10105,France,23.45.67.89,23.45.67.80
10,Customer EEALV,"Culp, Scott",Accounting Manager,8901 Tsawassen Blvd.,Tsawassen,BC,10111,Canada,(604) 901-2345,(604) 678-9012


21. Check option, to check if our insertion satisfies the views query. What if we insert a UK customer into the USACusts View? It'll leak into the main table without anyone knowing.

In [81]:
INSERT INTO Sales.USACusts(
  companyname, contactname, contacttitle, address,
  city, region, postalcode, country, phone, fax)
 VALUES(
  N'Customer ABCDE', N'Contact ABCDE', N'Title ABCDE', N'Address ABCDE',
  N'London', NULL, N'12345', N'UK', N'012-3456789', N'012-3456789');
--notice we're adding rows that dont satisfy the views query
--we wont see it in the view, but the actual table

SELECT custid, companyname, country
FROM Sales.USACusts
WHERE companyname = N'Customer ABCDE';

SELECT custid, companyname, country
FROM Sales.Customers
WHERE companyname = N'Customer ABCDE'

(1 row affected)

(1 row affected)

(8 rows affected)

Total execution time: 00:00:00.046

custid,companyname,country
97,Customer ABCDE,USA


custid,companyname,country
92,Customer ABCDE,UK
93,Customer ABCDE,UK
94,Customer ABCDE,UK
95,Customer ABCDE,UK
96,Customer ABCDE,UK
97,Customer ABCDE,USA
98,Customer ABCDE,UK
99,Customer ABCDE,UK


In [84]:
ALTER VIEW Sales.USACusts WITH SCHEMABINDING
AS

SELECT
  custid, companyname, contactname, contacttitle, address,
  city, region, postalcode, country, phone, fax
FROM Sales.Customers
WHERE country = N'USA'
WITH CHECK OPTION;
GO

INSERT INTO Sales.USACusts(
  companyname, contactname, contacttitle, address,
  city, region, postalcode, country, phone, fax)
 VALUES(
  N'Customer FGHIJ', N'Contact FGHIJ', N'Title FGHIJ', N'Address FGHIJ',
  N'London', NULL, N'12345', N'UK', N'012-3456789', N'012-3456789');


Commands completed successfully.

: Msg 550, Level 16, State 1, Line 12
The attempted insert or update failed because the target view either specifies WITH CHECK OPTION or spans a view that specifies WITH CHECK OPTION and one or more rows resulting from the operation did not qualify under the CHECK OPTION constraint.

The statement has been terminated.

Total execution time: 00:00:00.064

In [86]:
DELETE FROM Sales.Customers
WHERE custid > 91;

DROP VIEW IF EXISTS Sales.USACusts;
GO

(0 rows affected)

Total execution time: 00:00:00.015

22. **GetCustOrders function.** Takes customer id as argument and returns a table of the customers orders. **Get order tables for customer 1 and 2.**

In [91]:
use Northwinds2022TSQLV7

drop function if exists dbo.GetCustOrders
go

create function dbo.GetCustOrders
    (@cid as int) returns table 
as 
return --actual return table derived from select statement
    select *
    from Sales.[Order]
    where CustomerId = @cid;
go

--using the function
select CustomerId, OrderId
from dbo.GetCustOrders(1) as O1

select CustomerId, OrderId
from dbo.GetCustOrders(2) as O2



Commands completed successfully.

Commands completed successfully.

(6 rows affected)

(4 rows affected)

Total execution time: 00:00:00.062

CustomerId,OrderId
1,10643
1,10692
1,10702
1,10835
1,10952
1,11011


CustomerId,OrderId
2,10308
2,10625
2,10759
2,10926


23) **Return table of customer 1s orders and matching order details.**

In [94]:
use Northwinds2022TSQLV7;

select O.OrderId, O.CustomerId, OD.ProductId, OD.Quantity
from dbo.GetCustOrders(1) as O
    inner join
    Sales.[OrderDetail] as OD
        on O.OrderId = OD.OrderId
go

(12 rows affected)

Total execution time: 00:00:00.034

OrderId,CustomerId,ProductId,Quantity
10643,1,28,15
10643,1,39,21
10643,1,46,2
10692,1,63,20
10702,1,3,6
10702,1,76,15
10835,1,59,15
10835,1,77,2
10952,1,6,16
10952,1,28,2


In [95]:
use Northwinds2022TSQLV7

drop function if exists dbo.GetCustOrders
go

Commands completed successfully.

Total execution time: 00:00:00.018

24. **All combinations of shipper ids and customer ids.**

In [96]:
use Northwinds2022TSQLV7

select S.ShipperId, E.EmployeeId
from Sales.[Shipper] as S
    cross join 
    HumanResources.[Employee] AS E

(27 rows affected)

Total execution time: 00:00:00.085

ShipperId,EmployeeId
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
2,1


25. Apply. **Returns all combinations of shipper id and employee id.** Rather than cross join, it calculates the Employee table for each row and combines the two.

In [97]:
use Northwinds2022TSQLV7

select S.ShipperId, E.EmployeeId
from Sales.[Shipper] as S
    cross apply 
    HumanResources.[Employee] AS E

(27 rows affected)

Total execution time: 00:00:00.018

ShipperId,EmployeeId
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
2,1


26. **3 most recent orders for each customer.** For each implies cross apply.

In [8]:
use Northwinds2022TSQLV7
 
select C.CustomerId, A.OrderId, A.OrderDate
from Sales.[Customer] as C
    cross apply (
        select top (3) OrderId, EmployeeId, OrderDate, RequiredDate
        from Sales.[Order] as O
        where O.CustomerId = C.CustomerId
        order by orderdate desc, orderid desc
    ) as A

(263 rows affected)

Total execution time: 00:00:00.061

CustomerId,OrderId,OrderDate
1,11011,2016-04-09
1,10952,2016-03-16
1,10835,2016-01-15
2,10926,2016-03-04
2,10759,2015-11-28
2,10625,2015-08-08
3,10856,2016-01-28
3,10682,2015-09-25
3,10677,2015-09-22
4,11016,2016-04-10
